# Code for importing finometer txt data

## part 1

This package was written blah 

In [3]:
import pandas as pd
import numpy as np
import pathlib as pl
import datetime as dt
from functools import reduce #needed to allow merging of multiple datasets      

home = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data')
p1 = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 1_2022-10-27_09.07.37')
p2 = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Partcipant 59_2022-11-21_10.37.29')
time_stamps = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Timesheets.csv')

In [4]:
# Some instances of 'Participant' are misspelled as 'Partcipant' in the file names. This is a quick fix to correct this.
# Define the directory path where you want to replace the string
directory = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data')

# Define the old string and the new string
old_str = 'Partcipant'
new_str = 'Participant'

# Walk through all directories and files within the specified directory
for path in directory.glob('**/*'):
    if old_str in path.name:
        # Replace the old string with the new string in the file or directory name
        new_name = path.name.replace(old_str, new_str)
        if new_name != path.name:
            # If the name has changed, rename the file or directory
            path.rename(path.with_name(new_name))
            print(f'Renamed {path.name} to {new_name}')


In [5]:
def import_finometer_intervals_feb_23(folder_path, interval='1T', save_csv=False):
    '''This function imports the finometer data and then calculates the average of each measure over the selected time period'''

    if not isinstance(folder_path, pl.Path): # check if folder_path is a pathlib.Path object
        raise TypeError('folder_path must be a pathlib.Path object')
    elif not folder_path.exists() or not folder_path.is_dir(): # check if folder_path exists and is a directory
        raise ValueError('folder_path does not exist or is not a directory')
    else:
        # find the .txt file in the folder
        files = [file for file in folder_path.glob('*.txt')]
        if len(files) != 1:
            raise ValueError(f'Expected one .txt file, but found {len(files)} in the folder')
        file = files[0]

        # read in the data from the .txt file
        df = pd.read_csv(
            file,
            sep=';',
            header=0,
            skiprows=8,
            skipfooter=1,
            engine='python',
        )

        # drop the unnamed column at position 13
        df = df.drop(df.columns[13], axis=1)

        # set the 'Time (s)' column to a datetime object
        df['Time (s)'] = pd.to_datetime(df['Time (s)'], format='%H:%M:%S.%f')

        # set the 'Time (s)' column as the index and resample the DataFrame
        df = df.set_index('Time (s)').resample(interval).mean()

        # format the DatetimeIndex to only show hours and minutes
        # df.index = df.index.strftime('%H:%M')

        if save_csv:
            df.to_csv(folder_path / f'imported data for {file.stem.split("_")[0]}.csv', index=True)
            print(f'CSV saved for {file.stem.split("_")[0]}')

        return df


In [6]:
# This cell creates the 1 minute intervals for each participant
# it saves a file for each person in their folder
[import_finometer_intervals_feb_23(i, save_csv=True) for i in home.iterdir() if i.is_dir()]

C:\Users\kevin.omalley\AppData\Local\Temp\ipykernel_13100\1077607933.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.set_index('Time (s)').resample(interval).mean()
C:\Users\kevin.omalley\AppData\Local\Temp\ipykernel_13100\1077607933.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.set_index('Time (s)').resample(interval).mean()
C:\Users\kevin.omalley\AppData\Local\Temp\ipykernel_13100\1077607933.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or selec

CSV saved for Participant 11
CSV saved for Participant 1
CSV saved for Participant 59
CSV saved for Participant 60
CSV saved for Participant 64


C:\Users\kevin.omalley\AppData\Local\Temp\ipykernel_13100\1077607933.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.set_index('Time (s)').resample(interval).mean()
C:\Users\kevin.omalley\AppData\Local\Temp\ipykernel_13100\1077607933.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.set_index('Time (s)').resample(interval).mean()


[                     Systolic Pressure (mmHg)  Diastolic Pressure (mmHg)  \
 Time (s)                                                                   
 1900-01-01 10:44:00                118.551020                  66.428571   
 1900-01-01 10:45:00                168.488636                 102.215909   
 1900-01-01 10:46:00                162.258065                  97.774194   
 1900-01-01 10:47:00                130.155172                  77.586207   
 1900-01-01 10:48:00                136.615385                  78.747253   
 1900-01-01 10:49:00                130.098901                  76.923077   
 1900-01-01 10:50:00                131.000000                  76.456522   
 1900-01-01 10:51:00                133.400000                  79.242105   
 1900-01-01 10:52:00                139.865169                  81.786517   
 1900-01-01 10:53:00                132.044944                  77.123596   
 1900-01-01 10:54:00                129.066667                  77.866667   

In [7]:
# This function imports the protocol times from the .csv file and returns a dataframe with the protocol times for each participant as rows
# it has to 'clean' the data a little as well, but if we get batter at planning studies with this in mind it should be easier and maybe this can be modified
def import_protocol_times(times_file_path, add_seconds=False, flatten_seconds=False, save_csv=False):
    '''This function imports the protocol times from the .csv file and returns a dataframe with the protocol times for each participant
    
    file_path: pathlib.Path object
        The path to the .csv file containing the protocol times
    add_seconds: boolean (optional)
        If True, seconds will be added to the time values (if missing)
    save_csv: boolean (optional)
        If True, the imported data will be saved as a .csv file in the same folder as the .csv file, this is not always needed and should be used sparingly
    flatten_seconds: boolean (optional)
        If True, seconds will be set to 00 for all time values
    '''

    def add_seconds_to_time(time_str):
        '''This function adds seconds to the time string for in case the time string is missing seconds'''
        if len(time_str) == 5:
            time_str += ":00"
        return time_str

    def flatten_seconds(time_str):
        '''This function sets seconds to 00 for a given time string'''
        return time_str[:5] + ':00'

    if not isinstance(times_file_path, pl.Path):#check if folder_path is a pathlib.Path object
        raise TypeError('file_path must be a pathlib.Path object')
    elif not times_file_path.exists(): #  and if it exists
        raise ValueError('file_path does not exist')
    elif not times_file_path.is_file(): #  and is a file 
        raise ValueError('file_path is not a file')
    elif times_file_path.suffix != '.csv': #  and is a csv file
        raise ValueError('file_path is not an csv file')
    else:
        df = pd.read_csv(times_file_path, delimiter= ',')
        df.columns = [col.strip() for col in df.columns]
        cols_to_keep = ['Participant ID', 'Start of Baseline', 'End of Baseline', 'Start of Task 1', 'End of Task 1', 'Start of Recovery Period', 'End of Recovery Period']
        df = df[cols_to_keep].applymap(lambda x: str(x).strip('"') if isinstance(x, str) else x)
        
        if add_seconds and flatten_seconds:
            raise ValueError('Only one of add_seconds and flatten_seconds can be True')

        if add_seconds:
            try:
                df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: add_seconds_to_time(x) if isinstance(x, str) else x)
                df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: pd.to_datetime(x, format='%H:%M:%S', errors='coerce'))
            except:
                print('Could not add seconds to time, please check the time format')

        elif flatten_seconds:
            try:
                df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: add_seconds_to_time(x) if isinstance(x, str) else x)
                df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: flatten_seconds(x) if isinstance(x, str) else x)
                df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: pd.to_datetime(x, format='%H:%M:%S', errors='coerce'))
            except:
                print('Could not set seconds to 00, please check the time format')
        
        if save_csv: #if you want to save the csv file (which may be useful if you want to use the data in other ways)
            try:
                df.to_csv(times_file_path.parent / f"cleaned times.csv", index=False)
                print(f"CSV saved for {times_file_path.stem}")
            except Exception as e:
                print(f"Could not save csv file, error: {e}")
            


        return df

In [8]:
# This cell imports the protocol times from the .csv file
y = import_protocol_times(
    time_stamps,
    flatten_seconds = True,
    save_csv=True
    )

CSV saved for Timesheets


In [9]:
# this is the function that reshapes the data from the imported files into a format that can be used for the analysis
def import_protocol_averages_feb_23(file_path, t1=None, t2=None, t3=None, t4=None, t5=None, t6=None, save_csv=False):
    '''A function that imports the averaged finomter files (which have already been processed from the raw data)
    to produce averages for each section of the experiment protocol'''

    if not isinstance(file_path, pl.Path):
        raise TypeError('file_path must be a pathlib.Path object')
    elif not file_path.exists():
        raise ValueError('file_path does not exist')
    else:
        # check if the file is a csv or xlsx file
        if file_path.suffix == '.csv':
            try:
                intervals = pd.read_csv(file_path, index_col=0, parse_dates=True)
            except pd.errors.EmptyDataError:
                print(f"File {file_path} is empty")
        elif file_path.suffix == '.xlsx':
            try:
                intervals = pd.read_excel(file_path, index_col=0, parse_dates=True)
            except pd.errors.EmptyDataError:
                print(f"File {file_path} is empty")
        else:
            raise ValueError("File must be a Path object for a .csv or .xlsx file")

    intervals.index = pd.to_datetime(intervals.index).strftime('%H:%M:%S')
    intervals = intervals.replace('--', np.nan)
    intervals = intervals.dropna(how='all')

    def create_protocol_part_df(frame, start, end, tag, filename):
        '''A function that creates a dataframe representing a time period within a particular experimental protocol from a given DataFrame.
        Takes a DataFrame, start and end times for the portion of the experimental protocol.
        Returns a DataFrame with the mean values of the given columns during that portion of the study.'''
        period = frame.loc[start:end].dropna().T
        period['Average'] = period.mean(axis=1)
        b = pd.DataFrame(period['Average']).T
        b.columns = [f'{i}_{tag}' for i in b.columns]
        b['Participant ID'] = f'{filename}'
        return b

    # creating the baseline dataframe
    baseline = create_protocol_part_df(intervals, t1 , t2, 'bl', file_path.parent.stem.split('_')[0])

    # creating the task dataframe
    task = create_protocol_part_df(intervals, t3, t4, 'task', file_path.parent.stem.split('_')[0])

    # creating the recovery dataframe
    recovery = create_protocol_part_df(intervals, t5, t6, 'rec', file_path.parent.stem.split('_')[0])

    # creating a dataframe from the smaller dataframes
    data_merge = reduce(lambda left, right: pd.merge(left , right, on = ["Participant ID"], how = "outer"), [baseline, task, recovery])
    data_merge.set_index('Participant ID', inplace=True)

    if save_csv:
        data_merge.to_csv(file_path.parent / f"{file_path.parent.stem.split('_')[0]} protocol_averages.csv")
        print(f"Saved {file_path.parent.stem.split('_')[0]} protocol averages.csv to {file_path.parent}")
    
    return data_merge

In [10]:

# this cell defines the function that makes analysable dataframes from the imported data
def make_protocol_rows(data_path, timestamps_df):
    """
    Import data for participants whose timestamps are all non-null.
    
    Parameters:
    - data_path: pl.Path or str, the path to the folder containing the data files
    - timestamps_df: pandas DataFrame, containing the timestamps for each participant
    
    Returns:
    - None
    """
    data = pl.Path(data_path)
    for n in timestamps_df.index:
        if timestamps_df.iloc[n, 1:].isnull().values.any():
            print(f"Skipping participant {timestamps_df.loc[n, 'Participant ID']} due to missing timestamps")
            continue
        participant_id = timestamps_df.loc[n, 'Participant ID']
        file_name = f'imported data for {participant_id}.csv'
        file_paths = list(data.glob(f'**/{file_name}'))
        if not file_paths:
            print(f'no file found for {participant_id}')
        else:
            full_path = file_paths[0].resolve()
            print(f'File {full_path} found')

            start_bl = timestamps_df.loc[n, 'Start of Baseline']
            end_bl = timestamps_df.loc[n, 'End of Baseline']

            start_task = timestamps_df.loc[n, 'Start of Task 1']
            end_task = timestamps_df.loc[n, 'End of Task 1']

            start_rec = timestamps_df.loc[n, 'Start of Recovery Period']
            end_rec = timestamps_df.loc[n, 'End of Recovery Period']

            import_path = f'imported data for {participant_id}.csv'

            import_protocol_averages_feb_23(full_path,
                                            t1=start_bl.strftime('%H:%M:%S'),
                                            t2=end_bl.strftime('%H:%M:%S'),
                                            t3=start_task.strftime('%H:%M:%S') if not pd.isna(start_task) else None,
                                            t4=end_task.strftime('%H:%M:%S') if not pd.isna(end_task) else None,
                                            t5=start_rec.strftime('%H:%M:%S') if not pd.isna(start_rec) else None,
                                            t6=end_rec.strftime('%H:%M:%S') if not pd.isna(end_rec) else None,
                                            save_csv=True)


In [11]:
make_protocol_rows(home, y)

File C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 1_2022-10-27_09.07.37\imported data for Participant 1.csv found
Saved Participant 1 protocol averages.csv to C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 1_2022-10-27_09.07.37
no file found for Participant 2
no file found for Participant 3
no file found for Participant 4
no file found for Participant 5
no file found for Participant 6
no file found for Participant 7
no file found for Participant 8
no file found for Participant 9
no file found for Participant 10
File C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 11_2022-11-02_10.44.24\imported data for Participant 11.csv found
Saved Participant 11 protocol averages.csv to C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 11_2022-11-02_10.44.24
no file found for Participant 12
no fil

In [12]:
dfs = []

# Loop over each csv file in the data directory
for csv_file in home.glob('**/*protocol_averages.csv'):
    # Read in the csv file as a DataFrame and append to the list
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate the list of DataFrames into a single DataFrame
merged = pd.concat(dfs, ignore_index=True)

# Print the merged DataFrame
merged

,Participant ID,Systolic Pressure (mmHg)_bl,Diastolic Pressure (mmHg)_bl,Mean Pressure (mmHg)_bl,Heart rate (bpm)_bl,Stroke Volume (ml)_bl,Left Ventricular Ejection Time (ms)_bl,Pulse Interval (ms)_bl,Maximum Slope (mmHg/s)_bl,Cardiac Output (l/min)_bl,...,Diastolic Pressure (mmHg)_rec,Mean Pressure (mmHg)_rec,Heart rate (bpm)_rec,Stroke Volume (ml)_rec,Left Ventricular Ejection Time (ms)_rec,Pulse Interval (ms)_rec,Maximum Slope (mmHg/s)_rec,Cardiac Output (l/min)_rec,Total Peripheral Resistance Medical Unit (mmHg.min/l)_rec,Total Peripheral Resistance CGS (dyn.s/cm5)_rec
0,Participant 11,130.332879,78.108381,99.450939,91.209037,45.424908,256.072941,663.801148,1462.956481,4.129134,...,72.010811,91.448227,93.397224,45.754887,254.769242,647.586505,1432.147818,4.261211,1.303526,1738.034860
1,Participant 1,200.833785,158.117934,175.274722,93.636373,16.872201,243.055101,645.721243,1107.788169,1.574826,...,165.978901,184.247640,87.168559,15.815396,242.591596,695.540413,1219.716616,1.376907,8.365026,11153.373913
2,Participant 59,127.249536,85.151100,101.121819,81.033436,50.088474,279.067656,754.299231,1094.034108,4.038792,...,96.629670,114.452219,76.781248,45.835289,285.152028,809.088194,1269.090986,3.513903,2.173602,2898.119207
3,Participant 60,130.142323,78.793088,99.122270,77.914501,76.740840,265.993014,774.847624,1319.679006,5.954827,...,85.491772,107.425855,75.680162,77.856399,276.553843,802.088157,1505.189827,5.852221,1.105452,1473.942915


In [18]:
def import_finometer_demographics(file_path):
    '''
    A function to import the demographic data from a smartmedical/finopress
    finometer .txt file
    
    file_path =  should be a path to a .txt file, this can be string but should preferably be a pathlib object to allow for use on multiple operating
    systems
    '''

    if not isinstance(file_path, pl.PurePath):
        try:
            file_path = pl.Path(file_path)
        except TypeError:
            raise TypeError("""The 'file_path' must be a path object or a string that can be converted to a path object. Remember 'raw strings on windows devices'""")
    
    try:
        with open(file_path, 'r') as file:
            data=[]
            for line in file:
                if line.startswith('Identification'):
                    data.append(line.strip().split(';'))
                    next_line = next(file).strip()
                    if next_line:
                        data.append(next_line.split(';'))
                    break
            
            if len(data) < 2:
                raise ValueError("File doesn't contain expected data")
            

            df = pd.DataFrame(data, columns= ['Participant ID', 'ID 2', 'Age (yrs)', 'Height (cm)', 'Weight (kg)', 'Gender', 'Procedure', 'Model number'])
            df['Participant ID'] = df['Participant ID'].str.replace('Partcipant', 'Participant').str.strip()
            df = df.drop(0)
            df = df.drop('ID 2', axis = 1)


            return df      
                        

    except (TypeError, ValueError, FileNotFoundError) as err:
        print(str(file_path), type(err), err)
        return None


In [19]:
demographics = pd.concat([import_finometer_demographics(f) for f in p1.parent.glob("**/*") if f.suffix == '.txt'], axis = 0, ignore_index=True)


C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\qualtrics columns.txt <class 'ValueError'> File doesn't contain expected data


In [20]:
demographics

,Participant ID,Age (yrs),Height (cm),Weight (kg),Gender,Procedure,Model number
0,Participant 11,18,168,53,Female,,9715
1,Participant 1,18,150,48,Female,,9715
2,Participant 59,20,162,61,Female,,9715
3,Participant 60,18,168,72,Female,,9715
4,Participant 64,18,164,57,Female,,9715


In [21]:
demo_merged = pd.merge(demographics, merged, on = 'Participant ID', how = 'outer')

In [22]:
demo_merged

,Participant ID,Age (yrs),Height (cm),Weight (kg),Gender,Procedure,Model number,Systolic Pressure (mmHg)_bl,Diastolic Pressure (mmHg)_bl,Mean Pressure (mmHg)_bl,...,Diastolic Pressure (mmHg)_rec,Mean Pressure (mmHg)_rec,Heart rate (bpm)_rec,Stroke Volume (ml)_rec,Left Ventricular Ejection Time (ms)_rec,Pulse Interval (ms)_rec,Maximum Slope (mmHg/s)_rec,Cardiac Output (l/min)_rec,Total Peripheral Resistance Medical Unit (mmHg.min/l)_rec,Total Peripheral Resistance CGS (dyn.s/cm5)_rec
0,Participant 11,18,168,53,Female,,9715,130.332879,78.108381,99.450939,...,72.010811,91.448227,93.397224,45.754887,254.769242,647.586505,1432.147818,4.261211,1.303526,1738.034860
1,Participant 1,18,150,48,Female,,9715,200.833785,158.117934,175.274722,...,165.978901,184.247640,87.168559,15.815396,242.591596,695.540413,1219.716616,1.376907,8.365026,11153.373913
2,Participant 59,20,162,61,Female,,9715,127.249536,85.151100,101.121819,...,96.629670,114.452219,76.781248,45.835289,285.152028,809.088194,1269.090986,3.513903,2.173602,2898.119207
3,Participant 60,18,168,72,Female,,9715,130.142323,78.793088,99.122270,...,85.491772,107.425855,75.680162,77.856399,276.553843,802.088157,1505.189827,5.852221,1.105452,1473.942915
4,Participant 64,18,164,57,Female,,9715,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
